In [2]:
from tensorflow.keras.models import *
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

In [35]:
def unet_valid(pretrained_weights = None,input_size = (572,572,4),output_size = 3):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) # 284

    conv1 = tf.keras.layers.Cropping2D(cropping=((88,88), (88,88)))(conv1) # 568 -> 392

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) # 140

    conv2 = tf.keras.layers.Cropping2D(cropping=((40,40), (40,40)))(conv2) # 280 -> 200

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv3)
   
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3) # 68
  
    conv3 = tf.keras.layers.Cropping2D(cropping=((16,16), (16,16)))(conv3) # 136 -> 104
   
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv4)
    
    drop4 = Dropout(0.5)(conv4)

    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4) # 32
   
    conv4 = tf.keras.layers.Cropping2D(cropping=((4,4), (4,4)))(drop4) # 64 -> 56

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(pool4)
    
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv5)

    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(conv9)

    conv10 = Conv2D(output_size, (1,1), activation = 'softmax')(conv9)

#     model = Model(input = inputs, output = conv10)

#     changed to work in TF2.0
    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics=['accuracy',MeanIoU( num_classes=output_size)])
    
#  loss : mse / categorical_crossentropy sparse_categorical_crossentropy
#  metrics: MeanIoU(num_classes=4)

#     model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

model = unet_valid()

NameError: name 'MeanIoU' is not defined

In [ ]:
[(None, 64, 64, 512), (None, 55, 55, 512)]

In [6]:
import tensorflow as tf
BATCH_SIZE = 1
NUM_BOXES = 5
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
CHANNELS = 3
CROP_SIZE = (24, 24)

image = tf.random.normal(shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH,
CHANNELS) )
print(image.shape)
boxes = tf.random.uniform(shape=(NUM_BOXES, 4))
box_indices = tf.random.uniform(shape=(NUM_BOXES,), minval=0,
maxval=BATCH_SIZE, dtype=tf.int32)
output = tf.image.crop_and_resize(image, boxes, box_indices, CROP_SIZE)
output.shape  #=> (5, 24, 24, 3)

(1, 256, 256, 3)


TensorShape([5, 24, 24, 3])

In [9]:
x = [[[1.0, 2.0, 3.0],
      [4.0, 5.0, 6.0],
      [7.0, 8.0, 9.0],
      [10.0, 11.0, 12.0]],
    [[13.0, 14.0, 15.0],
      [16.0, 17.0, 18.0],
      [19.0, 20.0, 21.0],
      [22.0, 23.0, 24.0]],
    [[25.0, 26.0, 27.0],
      [28.0, 29.0, 30.0],
      [31.0, 32.0, 33.0],
      [34.0, 35.0, 36.0]],
    [[37.0, 38.0, 39.0],
      [40.0, 41.0, 42.0],
      [43.0, 44.0, 45.0],
      [46.0, 47.0, 48.0]]]
y = tf.image.central_crop(x, 0.5)

In [13]:
len(x[0])

4

In [14]:
len(y[0])

2

In [16]:
import numpy as np
input_shape = (2, 28, 28, 3)
x = np.arange(np.prod(input_shape)).reshape(input_shape)
y = tf.keras.layers.Cropping2D(cropping=((2, 2), (4, 4)))(x)
print(y.shape)

(2, 24, 20, 3)
